In [27]:
%run ../notebooks/Utilities.ipynb

In [31]:
#Setting the schema of the datasets.

schema_hired_employees = StructType ([
    StructField("id",IntegerType(), False),
    StructField("name",StringType(), True),
    StructField("datetime",StringType(), True),
    StructField("department_id",IntegerType(), True),
    StructField("job_id",IntegerType(), True)
])

schema_departments = StructType ([
    StructField("id",IntegerType(), False),
    StructField("department",StringType(), True)
])

schema_jobs = StructType ([
    StructField("id",IntegerType(), False),
    StructField("job",StringType(), True)   
])

In [32]:
# Load CSV files
df_hired_employees = spark.read.csv("../data/hired_employees.csv", schema=schema_hired_employees)
df_departments = spark.read.csv("../data/departments.csv", schema=schema_departments)
df_jobs = spark.read.csv("../data/jobs.csv", schema=schema_jobs)

#Format the datetime column as datetime ISO formart
df_hired_employees = df_hired_employees.withColumn('datetime',date_format('datetime','yyyy-MM-dd'))

In [33]:
df_hired_employees.show()

+---+------------------+----------+-------------+------+
| id|              name|  datetime|department_id|job_id|
+---+------------------+----------+-------------+------+
|  1|       Harold Vogt|2021-11-06|            2|    96|
|  2|          Ty Hofer|2021-05-30|            8|  NULL|
|  3|       Lyman Hadye|2021-09-01|            5|    52|
|  4|     Lotti Crowthe|2021-10-01|           12|    71|
|  5|    Gretna Lording|2021-10-10|            6|    80|
|  6|    Marlow Antecki|2021-04-23|            6|    95|
|  7|      Joan Rillett|2021-10-09|            9|    78|
|  8|  Ulrick Nucciotti|2021-07-23|            8|   169|
|  9|Lucretia Northcote|2021-04-01|            9|     8|
| 10|      Arty Giacobo|2022-02-08|            6|    62|
| 11|      Libbi Dowtry|2021-07-04|            6|    41|
| 12|      Jacky Oldred|2021-06-20|            7|    29|
| 13|      Raine Mowett|2021-12-23|            8|    83|
| 14|   Melonie Slocomb|2021-12-30|           12|   121|
| 15|    Robers Swinden|2021-02